# 2019 Airline Delays w/Weather and Airport Detail

## Importation of all the packages we will use for this project

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Preparation of the data

#### Concatenation of the 12 files representing each month in one dataframe

In [2]:
src_path = 'raw_data/ONTIME_REPORTING_'
data = pd.DataFrame()

for i in range(1, 13):
    end_path = ('0' + str(i) if i < 10 else str(i)) + '.csv'
    print('> reading from', src_path + end_path)
    df = pd.read_csv(src_path + end_path)
    data = pd.concat([data, df])

> reading from raw_data/ONTIME_REPORTING_01.csv
> reading from raw_data/ONTIME_REPORTING_02.csv
> reading from raw_data/ONTIME_REPORTING_03.csv
> reading from raw_data/ONTIME_REPORTING_04.csv
> reading from raw_data/ONTIME_REPORTING_05.csv
> reading from raw_data/ONTIME_REPORTING_06.csv
> reading from raw_data/ONTIME_REPORTING_07.csv
> reading from raw_data/ONTIME_REPORTING_08.csv
> reading from raw_data/ONTIME_REPORTING_09.csv
> reading from raw_data/ONTIME_REPORTING_10.csv
> reading from raw_data/ONTIME_REPORTING_11.csv
> reading from raw_data/ONTIME_REPORTING_12.csv


In [3]:
data.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 32
0,1,6,7,9E,N8694A,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,37.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,1,7,1,9E,N8970D,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,32.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN
2,1,8,2,9E,N820AY,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,39.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN
3,1,9,3,9E,N840AY,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,37.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10,4,9E,N8969A,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,41.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN


#### Keeping only the usefull columns for our study

In [4]:
data = data[['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_NAME', 'DEP_DELAY_NEW', 'CANCELLED']]

In [5]:
data.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,DEP_DELAY_NEW,CANCELLED
0,1,6,7,10397,"Atlanta, GA",0.0,0.0
1,1,7,1,10397,"Atlanta, GA",0.0,0.0
2,1,8,2,10397,"Atlanta, GA",0.0,0.0
3,1,9,3,10397,"Atlanta, GA",0.0,0.0
4,1,10,4,10397,"Atlanta, GA",0.0,0.0


##### Adding the missing columns for our study

In [7]:
def formatDate(df):
    month = '0' + str(df['MONTH']) if df['MONTH'] < 10 else str(df['MONTH'])
    day = '0' + str(df['DAY_OF_MONTH']) if df['DAY_OF_MONTH'] < 10 else str(df['DAY_OF_MONTH'])
    return '2019-' + month + '-' + day
data['DATE'] = data.apply(formatDate, axis=1)

In [8]:
data

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,DEP_DELAY_NEW,CANCELLED,DATE
0,1,6,7,10397,"Atlanta, GA",0.0,0.0,2019-01-06
1,1,7,1,10397,"Atlanta, GA",0.0,0.0,2019-01-07
2,1,8,2,10397,"Atlanta, GA",0.0,0.0,2019-01-08
3,1,9,3,10397,"Atlanta, GA",0.0,0.0,2019-01-09
4,1,10,4,10397,"Atlanta, GA",0.0,0.0,2019-01-10
...,...,...,...,...,...,...,...,...
625758,12,31,2,13204,"Orlando, FL",64.0,0.0,2019-12-31
625759,12,31,2,11278,"Washington, DC",0.0,0.0,2019-12-31
625760,12,31,2,14100,"Philadelphia, PA",0.0,0.0,2019-12-31
625761,12,31,2,10721,"Boston, MA",0.0,0.0,2019-12-31


In [9]:
airports = pd.read_csv('raw_data/airports_list.csv')
weather = pd.read_csv('raw_data/airport_weather_2019.csv')
weather_data = pd.merge(airports, weather, how='left', on='NAME')

In [10]:
weather_data = weather_data[['DATE', 'ORIGIN_AIRPORT_ID', 'PRCP', 'SNWD', 'TMAX', 'TMIN', 'TAVG', 'AWND']]
weather_data

,DATE,ORIGIN_AIRPORT_ID,PRCP,SNWD,TMAX,TMIN,TAVG,AWND
0,2019-01-01,12992,0.00,0.0,45.0,39.0,NaN,4.70
1,2019-01-02,12992,0.39,0.0,39.0,36.0,NaN,2.01
2,2019-01-03,12992,0.44,0.0,41.0,37.0,NaN,6.26
3,2019-01-04,12992,0.13,0.0,47.0,37.0,NaN,2.01
4,2019-01-05,12992,0.00,0.0,62.0,37.0,NaN,1.79
...,...,...,...,...,...,...,...,...
35020,2019-12-27,10713,0.00,0.0,35.0,26.0,32.0,5.82
35021,2019-12-28,10713,0.00,0.0,39.0,24.0,29.0,2.24
35022,2019-12-29,10713,0.04,0.0,32.0,28.0,31.0,6.26
35023,2019-12-30,10713,0.00,0.0,34.0,29.0,31.0,2.46


In [11]:
weather_data.isna().sum()

DATE                     1
ORIGIN_AIRPORT_ID        0
PRCP                    22
SNWD                 11565
TMAX                     6
TMIN                     9
TAVG                  7013
AWND                     9
dtype: int64

In [12]:
def avgTemperature(data):
    return round((data['TMIN'] + data['TMAX']) / 2, 1)

In [13]:
weather_data.drop(weather_data.loc[weather_data['DATE'].isna()].index, axis=0, inplace=True)
weather_data['PRCP'].fillna(round(weather_data.groupby('ORIGIN_AIRPORT_ID')['PRCP'].transform('mean'), 1), inplace=True)
weather_data['TMIN'].fillna(round(weather_data.groupby('ORIGIN_AIRPORT_ID')['TMIN'].transform('mean'), 1), inplace=True)
weather_data['TAVG'].fillna(round(weather_data.groupby('ORIGIN_AIRPORT_ID')['TAVG'].transform('mean'), 1), inplace=True)
weather_data['TAVG'].fillna(avgTemperature(weather_data), inplace=True)
weather_data['AWND'].fillna(round(weather_data.groupby('ORIGIN_AIRPORT_ID')['AWND'].transform('mean'), 1), inplace=True)
weather_data.fillna(0, inplace=True)

In [14]:
weather_data.isna().sum()

DATE                 0
ORIGIN_AIRPORT_ID    0
PRCP                 0
SNWD                 0
TMAX                 0
TMIN                 0
TAVG                 0
AWND                 0
dtype: int64

In [15]:
weather_data

,DATE,ORIGIN_AIRPORT_ID,PRCP,SNWD,TMAX,TMIN,TAVG,AWND
0,2019-01-01,12992,0.00,0.0,45.0,39.0,42.0,4.70
1,2019-01-02,12992,0.39,0.0,39.0,36.0,37.5,2.01
2,2019-01-03,12992,0.44,0.0,41.0,37.0,39.0,6.26
3,2019-01-04,12992,0.13,0.0,47.0,37.0,42.0,2.01
4,2019-01-05,12992,0.00,0.0,62.0,37.0,49.5,1.79
...,...,...,...,...,...,...,...,...
35020,2019-12-27,10713,0.00,0.0,35.0,26.0,32.0,5.82
35021,2019-12-28,10713,0.00,0.0,39.0,24.0,29.0,2.24
35022,2019-12-29,10713,0.04,0.0,32.0,28.0,31.0,6.26
35023,2019-12-30,10713,0.00,0.0,34.0,29.0,31.0,2.46


In [16]:
def formatDate2(df):
    if '/' in str(df['DATE']):
        month, day, year = df['DATE'].split('/')
        month = '0' + month if int(month) < 10 else month
        day = '0' + day if int(day) < 10 else day
        return year + '-' + month + '-' + day
    return df['DATE']

In [17]:
weather_data['DATE'] = weather_data.apply(formatDate2, axis=1)

In [18]:
weather_data['DATE'].head(2000)

0       2019-01-01
1       2019-01-02
2       2019-01-03
3       2019-01-04
4       2019-01-05
           ...    
1995    2019-06-20
1996    2019-06-21
1997    2019-06-22
1998    2019-06-23
1999    2019-06-24
Name: DATE, Length: 2000, dtype: object

In [19]:
all_data = pd.merge(data, weather_data, on=['DATE', 'ORIGIN_AIRPORT_ID'])
all_data.shape

(6694235, 14)

In [20]:
all_data

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,DEP_DELAY_NEW,CANCELLED,DATE,PRCP,SNWD,TMAX,TMIN,TAVG,AWND
0,1,6,7,10397,"Atlanta, GA",0.0,0.0,2019-01-06,0.00,0.0,69.0,43.0,53.0,6.49
1,1,6,7,10397,"Atlanta, GA",0.0,0.0,2019-01-06,0.00,0.0,69.0,43.0,53.0,6.49
2,1,6,7,10397,"Atlanta, GA",0.0,0.0,2019-01-06,0.00,0.0,69.0,43.0,53.0,6.49
3,1,6,7,10397,"Atlanta, GA",12.0,0.0,2019-01-06,0.00,0.0,69.0,43.0,53.0,6.49
4,1,6,7,10397,"Atlanta, GA",8.0,0.0,2019-01-06,0.00,0.0,69.0,43.0,53.0,6.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6694230,12,29,7,10299,"Anchorage, AK",0.0,0.0,2019-12-29,0.02,7.1,29.0,11.0,15.0,7.83
6694231,12,29,7,10299,"Anchorage, AK",0.0,0.0,2019-12-29,0.02,7.1,29.0,11.0,15.0,7.83
6694232,12,29,7,10299,"Anchorage, AK",0.0,0.0,2019-12-29,0.02,7.1,29.0,11.0,15.0,7.83
6694233,12,29,7,10299,"Anchorage, AK",0.0,0.0,2019-12-29,0.02,7.1,29.0,11.0,15.0,7.83


In [23]:
all_data = all_data.drop(['ORIGIN_AIRPORT_ID', 'DATE', 'TMAX'], axis=1)

In [24]:
all_data

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN_CITY_NAME,DEP_DELAY_NEW,CANCELLED,PRCP,SNWD,TMIN,TAVG,AWND
0,1,6,7,"Atlanta, GA",0.0,0.0,0.00,0.0,43.0,53.0,6.49
1,1,6,7,"Atlanta, GA",0.0,0.0,0.00,0.0,43.0,53.0,6.49
2,1,6,7,"Atlanta, GA",0.0,0.0,0.00,0.0,43.0,53.0,6.49
3,1,6,7,"Atlanta, GA",12.0,0.0,0.00,0.0,43.0,53.0,6.49
4,1,6,7,"Atlanta, GA",8.0,0.0,0.00,0.0,43.0,53.0,6.49
...,...,...,...,...,...,...,...,...,...,...,...
6694230,12,29,7,"Anchorage, AK",0.0,0.0,0.02,7.1,11.0,15.0,7.83
6694231,12,29,7,"Anchorage, AK",0.0,0.0,0.02,7.1,11.0,15.0,7.83
6694232,12,29,7,"Anchorage, AK",0.0,0.0,0.02,7.1,11.0,15.0,7.83
6694233,12,29,7,"Anchorage, AK",0.0,0.0,0.02,7.1,11.0,15.0,7.83


In [25]:
all_data.drop(all_data.loc[all_data['DEP_DELAY_NEW'].isna()].index, axis=0, inplace=True)

In [26]:
all_data.isna().sum()

MONTH               0
DAY_OF_MONTH        0
DAY_OF_WEEK         0
ORIGIN_CITY_NAME    0
DEP_DELAY_NEW       0
CANCELLED           0
PRCP                0
SNWD                0
TMIN                0
TAVG                0
AWND                0
dtype: int64

In [27]:
all_data.to_csv('delay_flights.csv', index=False)